# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [1]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [2]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import warnings

warnings.filterwarnings('ignore')

In [3]:
def build_classifiers():
    classifiers = []

    classifiers.append(LinearRegression())
    classifiers.append(KNeighborsClassifier())
    classifiers.append(SVC())
    classifiers.append(DecisionTreeClassifier())
    classifiers.append(GaussianNB())
    classifiers.append(QuadraticDiscriminantAnalysis())

    for classifier in classifiers:
        classifier.fit(data_set, labels)

    return classifiers

In [4]:
def build_stacked_classifier(classifiers, meta_classifier):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130, 3))

    # stacked classifier part:
    stacked_classifier = meta_classifier
    stacked_classifier.fit(output.reshape((130, 3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]), 3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [5]:
from itertools import permutations
from copy import deepcopy
import pandas as pd

classifiers = build_classifiers()
accuracies = []
seen_permutations = set()

for selected_classifiers in permutations(classifiers, 4):
    three_classifiers = selected_classifiers[:3]
    meta_classifier = deepcopy(selected_classifiers[-1])

    if isinstance(meta_classifier, LinearRegression):
        continue

    unique_permutation = tuple(sorted([str(element) for element in three_classifiers])) + tuple(str(meta_classifier))
    if unique_permutation not in seen_permutations:
        seen_permutations.add(unique_permutation)
        predicted = build_stacked_classifier(three_classifiers, meta_classifier)
        accuracy = accuracy_score(test_labels, predicted)
        accuracies.append((accuracy, str(three_classifiers[0]), str(three_classifiers[1]), str(three_classifiers[2]),
                           str(meta_classifier)))

sorted_accuracies = sorted(accuracies, key=lambda x: x[0], reverse=True)
df = pd.DataFrame(sorted_accuracies,
                  columns=['Accuracy', 'Classifier 1', 'Classifier 2', 'Classifier 3', 'Meta Classifier'])

df.head(15)

,Accuracy,Classifier 1,Classifier 2,Classifier 3,Meta Classifier
0,1.00,LinearRegression(),KNeighborsClassifier(),SVC(),GaussianNB()
1,1.00,LinearRegression(),KNeighborsClassifier(),DecisionTreeClassifier(),GaussianNB()
2,1.00,LinearRegression(),KNeighborsClassifier(),QuadraticDiscriminantAnalysis(),GaussianNB()
3,1.00,LinearRegression(),SVC(),DecisionTreeClassifier(),GaussianNB()
4,1.00,LinearRegression(),SVC(),QuadraticDiscriminantAnalysis(),GaussianNB()
5,1.00,LinearRegression(),DecisionTreeClassifier(),QuadraticDiscriminantAnalysis(),GaussianNB()
6,1.00,KNeighborsClassifier(),SVC(),DecisionTreeClassifier(),GaussianNB()
7,1.00,KNeighborsClassifier(),SVC(),QuadraticDiscriminantAnalysis(),GaussianNB()
8,1.00,KNeighborsClassifier(),DecisionTreeClassifier(),QuadraticDiscriminantAnalysis(),GaussianNB()
9,1.00,SVC(),DecisionTreeClassifier(),QuadraticDiscriminantAnalysis(),GaussianNB()


## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [6]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier


# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels


labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)


def calculate_error(model):
    predicted = model.predict(test_set)
    I = calculate_accuracy_vector(predicted, test_labels)
    Z = np.sum(I)
    return (1 + Z) / 1.0


def calculate_accuracy_vector(predicted, labels):
    result = []
    for i in range(len(predicted)):
        if predicted[i] == labels[i]:
            result.append(1)
        else:
            result.append(0)
    return result

Fill the two functions below:

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [7]:
def set_new_weights(model):
    I = np.array(calculate_accuracy_vector(model.predict(test_set), test_labels))
    return (1 + I) / (1 + I).sum()

Train the classifier with the code below:

In [8]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)

[0.00131839 0.00131839 0.0006592  0.00131839 0.0006592  0.00131839
 0.00131839 0.0006592  0.0006592  0.0006592  0.0006592  0.00131839
 0.00131839 0.0006592  0.00131839 0.00131839 0.0006592  0.00131839
 0.0006592  0.0006592  0.0006592  0.00131839 0.0006592  0.00131839
 0.00131839 0.0006592  0.0006592  0.00131839 0.00131839 0.0006592
 0.00131839 0.0006592  0.00131839 0.0006592  0.0006592  0.0006592
 0.0006592  0.0006592  0.0006592  0.0006592  0.00131839 0.0006592
 0.00131839 0.0006592  0.00131839 0.0006592  0.0006592  0.0006592
 0.0006592  0.00131839 0.00131839 0.0006592  0.00131839 0.00131839
 0.0006592  0.0006592  0.0006592  0.00131839 0.00131839 0.0006592
 0.00131839 0.00131839 0.00131839 0.0006592  0.0006592  0.0006592
 0.00131839 0.00131839 0.00131839 0.00131839 0.00131839 0.0006592
 0.00131839 0.00131839 0.0006592  0.00131839 0.0006592  0.0006592
 0.00131839 0.0006592  0.0006592  0.00131839 0.00131839 0.00131839
 0.00131839 0.00131839 0.0006592  0.0006592  0.00131839 0.00131839
 0.

Set the validation data set:

In [9]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [10]:
def get_prediction(x):
    output = np.array([classifier.predict(x) for classifier in classifiers])
    predicted = []
    for i in range(len(x)):
        classified = output[:, i]
        predicted.append(np.argmax(np.bincount(classified)))
    return predicted

Test it:

In [11]:
prediction = get_prediction(validate_x)[0]

print(prediction)

1
